## Strategisthub Sales Assistance

In [44]:
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.tools import tool
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
import os
from pathlib import Path
import bs4
from langchain_core.messages import SystemMessage
from dotenv import load_dotenv
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langgraph.checkpoint.memory import MemorySaver
from langchain.messages import HumanMessage

load_dotenv()

True

In [45]:
# helper functions

def load_pdfs_from_directory(directory_path: str):
    docs = []
    for pdf_file in Path(directory_path).rglob("*.pdf"):
        try:
            # print("Reading data from: ", pdf_file)
            loader = PyPDFLoader(str(pdf_file))
            docs.extend(loader.load())
        except Exception as e:
            print(f"Error loading {pdf_file}: {e}")
    return docs

# 2. Load data from websites
def load_from_websites(urls):
    docs = []
    for url in urls:
        loader = WebBaseLoader(url)
        docs.extend(loader.load())
    return docs

In [46]:
# import weaviate
# from weaviate.classes.init import Auth
# import os

# # Best practice: store your credentials in environment variables
# weaviate_url = os.environ["WEAVIATE_URL"]
# weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=weaviate_url,
#     auth_credentials=Auth.api_key(weaviate_api_key),
# )

# print(client.is_ready())  # Should print: `True`

# client.close() 

In [47]:
# 3. Combine and split documents
pdf_docs = load_pdfs_from_directory("/home/hp/Desktop/Workplace/CustomizeGPT/data")
web_urls = ["https://strategisthub.com/services/", "https://strategisthub.com/about/", "https://strategisthub.com/case-studies/", "https://strategisthub.com/blogs/"]
web_docs = load_from_websites(web_urls)

all_docs = pdf_docs + web_docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(all_docs)

In [62]:
# 4. Create vector store
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(doc_splits, embeddings)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [63]:
# Check the source or metadata of the content
def retrieve_documents(query: str):
    """Retrieve relevant documents."""
    docs = retriever.invoke(query)
    serialized = "\\n\\n".join(
        f"Source: {doc.metadata}\\nContent: {doc.page_content}"
        for doc in docs
    )
    return serialized, docs

ser,response = retrieve_documents("what are the major services of strategisthub?")

print(response[-1].metadata)
print(response[-1].page_content)

{'producer': 'iLovePDF', 'creator': 'Adobe Illustrator 26.0 (Windows)', 'creationdate': '2025-06-11T15:27:10+06:00', 'title': 'PitchDeck 2.0', 'moddate': '2025-06-11T10:33:07+00:00', 'source': '/home/hp/Desktop/Workplace/CustomizeGPT/data/Sales/Profiles & Portfolio Documents/Pitch Deck/PitchDeck.pdf', 'total_pages': 18, 'page': 1, 'page_label': '2'}
StrategistHub is a trusted technology partner that helps startups, 
SMBs, and enterprises  build scalable, high-performance digital 
solutions with the expertise of top-tier software engineers. With 
over a decade of experience in software development and
engineering services, we provide businesses with highly skilled 
engineers who bring deep technical expertise in AI-driven
solutions, custom software development, and advanced system 
architecture. Our team enables organizations to streamline


In [64]:
# tool node
# 5. Create retriever tool
@tool(response_format="content_and_artifact")
def retrieve_documents(query: str):
    """Retrieve relevant documents."""
    docs = retriever.invoke(query)
    serialized = "\\n\\n".join(
        f"Source: {doc.metadata}\\nContent: {doc.page_content}"
        for doc in docs
    )
    return serialized, docs

tools = [retrieve_documents]
tool_node = ToolNode(tools)

In [65]:
# 6. Enhanced system prompt for email responses
EMAIL_SYSTEM_PROMPT = """
You are a professional email assistant for our company's sales team. Your role is to respond to customer inquiries using ONLY information from our knowledge base.

CRITICAL RULES:
1. You MUST respond in proper business email format with subject line, salutation, body, and signature
2. If the customer's question can be answered using the provided context, write a helpful, professional email response
3. If the information is NOT in the knowledge base (context shows "NO_RELEVANT_INFORMATION_FOUND"), respond with a polite email explaining this
4. Never invent information or use external knowledge
5. Maintain a professional, helpful tone in all communications
6. Format your response as a ready-to-send email
7. Always start the subject with "Re: " followed by the original subject or an appropriate title

EMAIL FORMAT:
Subject: Re: [Original Subject or Appropriate Title]

Dear [Customer Name],

[Professional email body acknowledging their query and providing information or explaining limitations]

[Clear next steps or contact information if needed]

Best regards,
[Sale Team]
[Strategisthub]
"""

In [66]:
# 6. Build LangGraph nodes
model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

def call_model(state: MessagesState):
    response = model.invoke(
        [
            SystemMessage(
                        content=EMAIL_SYSTEM_PROMPT
                    )
        ]
        + state["messages"]
        )
    return {"messages": [response]}

def should_continue(state: MessagesState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"

# 7. Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge("__start__", "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

# app = workflow.compile()
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [67]:
# from IPython.display import Image, display
# display(Image(app.get_graph().draw_mermaid_png()))

In [68]:
user_input = f"""
from: john@exmaple.com

subject: need info about the Modernizing Global

Hi Sale Tean, i am john, i will be thankful if you can share some insights on the following topic:

'Modernizing Global Real Estate Experiences'

i will be waiting for your response

regards
john
"""

In [69]:
# memory = MemorySaver()
# app = workflow.compile(checkpointer=memory)

# Use with thread
config = {"configurable": {"thread_id": "1"}}
response = app.invoke(
    {"messages": [{"role": "user", "content": user_input}]},
    config
)
print(response["messages"][-1].content)

Subject: Re: need info about the Modernizing Global

Dear John,

Thank you for reaching out with your inquiry about "Modernizing Global Real Estate Experiences." 

Our collaboration with Keller Williams highlights key aspects of modernizing real estate technology. We focused on enhancing user experience by transitioning from a multi-page to a single-page CRM application, which significantly improves the experience for real estate agents. Additionally, we addressed the complexities of global expansion by transforming the entire suite and mobile applications to support multiple languages and currencies.

Key areas of focus included:
- Modernizing legacy infrastructure using microservices and micro frontends.
- Enabling multilingual and multi-currency support for global markets.
- Collaborating on ecosystem integration and cloud rollout to ensure long-term agility and growth.

If you would like more detailed information or have specific questions, please feel free to ask.

Best regards,  

In [39]:
config = {"configurable": {"thread_id": "1"}}
response = app.invoke(
    {"messages": [{"role": "user", "content": "what are the major services of strategisthub?"}]},
    config
)
print(response["messages"][-1].content)

Subject: Re: Major Services of StrategistHub

Dear [Customer Name],

Thank you for your inquiry regarding the major services offered by StrategistHub.

At StrategistHub, we pride ourselves on being a trusted technology partner for startups, small to medium-sized businesses (SMBs), and enterprises. Our primary services include:

1. **Custom Software Development**: We build scalable, high-performance digital solutions tailored to meet the unique needs of our clients.
2. **AI-Driven Solutions**: Our expertise in artificial intelligence allows us to create innovative solutions that enhance operational efficiency and user experiences.
3. **Advanced System Architecture**: We provide engineering excellence that drives digital transformation and long-term success for organizations.
4. **Engineering Services**: With a team of highly skilled engineers, we support businesses in streamlining operations and accelerating growth through innovative, future-ready solutions.

If you have any further que

In [ ]:
# """ Subject: Re: Inquiry About Major Services of Strategisthub
# Dear Customer,
# Thank you for your inquiry regarding the major services offered by Strategisthub. Unfortunately,
# I am unable to provide specific details about our services at this time, as the information is not available in our knowledge base.
# If you have any other questions or need assistance with a different topic, 
# please feel free to reach out.

# Best regards,
# Sales Team  \nStrategisthub"""

In [43]:
config = {"configurable": {"thread_id": "1"}}
response = app.invoke(
    {"messages": [{"role": "user", "content": "what is the vision and mission of strategisthub?"}]},
    config
)
print(response["messages"][-1].content)

Subject: Re: Vision and Mission of StrategistHub

Dear [Customer Name],

Thank you for your inquiry regarding the vision and mission of StrategistHub.

**Vision**: At StrategistHub, we envision a future where businesses thrive through intelligent innovation. Our goal is to become the trusted global partner for AI-driven digital transformation, helping businesses unlock their full potential, lead markets, and build scalable impact in a rapidly evolving tech landscape.

**Mission**: Our mission is to transform complex challenges into scalable opportunities through AI-powered strategies and deep technical expertise. We aim to empower organizations of all sizes to drive transformative growth and set new standards of excellence in their respective industries.

If you have any further questions or need additional information, please feel free to reach out.

Best regards,  
Sales Team  
Strategisthub


In [41]:
# from langchain.messages import HumanMessage
# result = app.invoke({
#     "messages": [HumanMessage(content=user_input)]
# })
# print(result["messages"][-1].content)